In [1]:
import pandas as pd


def data_loader(path):
    table = pd.read_csv(path, index_col = [0])
    return table

In [2]:
df = data_loader('./data/laptops.csv')

- df로 파일을 불러왔어

In [3]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [4]:
def slice_df(table, col_name):
    test_df = table[table[col_name].isnull()==True]
    train_df = table[table[col_name].isnull()==False]
    return test_df, train_df

In [5]:
test_df, train_df = slice_df(df, 'rating')

- 'rating' 이 NULL인 데이터프레임이랑 NOT NULL인 데이터프레임으로 나눠짐

In [6]:
def slice_sparses(*col_names):
    sparse_list = []
    for col in col_names:
        sparse_list.append(col)
    return sparse_list

In [7]:
def slice_denses(*col_names):
    dense_list = []
    for col in col_names:
        dense_list.append(col)
    return dense_list

- sparse column과 dense column을 받아서 리스트로 뱉어주고
- 그걸 변수로 다시 저장해야함

In [8]:
slice_sparse = slice_sparses('img_link','name','processor','ram','os','storage')

In [9]:
slice_dense = slice_denses('price(in Rs.)','display(in inch)')

In [10]:
slice_sparse

['img_link', 'name', 'processor', 'ram', 'os', 'storage']

In [11]:
def scaler_encoder(sparese, dense, target_df):
    target_df = target_df.copy()
    mms = MinMaxScaler(feature_range=(0, 1))
    target_df[dense] = mms.fit_transform(target_df[dense])
    for feat in sparese:
        lbe = LabelEncoder()
        target_df[feat] = lbe.fit_transform(target_df[feat])
        
    return target_df

In [12]:
train_df_transform = scaler_encoder(slice_sparse, slice_dense, train_df)


In [13]:
test_df_transform = scaler_encoder(slice_sparse, slice_dense, test_df)

In [14]:
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from deepctr.models import DeepFM

In [15]:
def fixlen_feature(target_df, slice_sparse_col, slice_dense_col):
    fixlen_feature_columns = [SparseFeat(feat, target_df[feat].max() + 1, embedding_dim=4) for feat in slice_sparse] +\
        [DenseFeat(feat, 1, ) for feat in slice_dense]
    return fixlen_feature_columns

In [16]:
train_fixlen_feature = fixlen_feature(train_df_transform, slice_sparse, slice_dense)

In [17]:
test_fixlen_feature = fixlen_feature(test_df_transform, slice_sparse, slice_dense)

In [52]:
def clone_fixlen_feature(fixlen_features):
    dnn_feature_col = fixlen_features
    linear_feature_col = fixlen_features
    return dnn_feature_col, linear_feature_col, get_feature_names(linear_feature_col + dnn_feature_col)

In [53]:
dnn_feature_col, linear_feature_col, train_last_feature = clone_fixlen_feature(train_fixlen_feature)

In [54]:
def train_test_split_pred(target_df, feature_names):
    train, test = train_test_split(target_df, test_size=0.2, random_state=2020)
    train_model_input = {name: train[name] for name in feature_names}
    test_model_input = {name: test[name] for name in feature_names}
    return train_model_input, test_model_input

In [55]:
train_fixlen_feature

[SparseFeat(name='img_link', vocabulary_size=401, embedding_dim=4, use_hash=False, vocabulary_path=None, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x000001D225920520>, embedding_name='img_link', group_name='default_group', trainable=True),
 SparseFeat(name='name', vocabulary_size=357, embedding_dim=4, use_hash=False, vocabulary_path=None, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x000001D225920FA0>, embedding_name='name', group_name='default_group', trainable=True),
 SparseFeat(name='processor', vocabulary_size=47, embedding_dim=4, use_hash=False, vocabulary_path=None, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x000001D22590FAC0>, embedding_name='processor', group_name='default_group', trainable=True),
 SparseFeat(name='ram', vocabulary_size=20, embedding_dim=4, use_has

In [63]:
train_pred_y = train_test_split_pred(train_df_transform, train_last_feature)

In [59]:
def make_model():
    DEFAULT_GROUP_NAME = "default_group"
    model = DeepFM(linear_feature_col, dnn_feature_col, fm_group=(DEFAULT_GROUP_NAME,), dnn_hidden_units=(256, 128, 64),
           l2_reg_linear=0.00001, l2_reg_embedding=0.00001, l2_reg_dnn=0, seed=1024, dnn_dropout=0,
           dnn_activation='relu', dnn_use_bn=False, task='regression')
    return model

In [60]:
model = make_model()

In [61]:
def model_compile(model ,loss_m, opt, metr):
    model = model.compile(loss = loss_m, optimizer = opt, metrics = [metr])


In [62]:
model_compile(model, 'mean_squared_error', 'adam', 'mse')

In [64]:
def model_fit(pred_y_data, target_value, epo):
    history = model.fit(pred_y, target_value, batch_size=256, epochs = epo, verbose=2, validation_split=0.2)
    return history

In [66]:
train_history = model_fit(train_pred_y, train_df_transform['rating'], 100)

Epoch 1/100
1/1 - 0s - loss: 0.0287 - mse: 0.0286 - val_loss: 0.6073 - val_mse: 0.6072
Epoch 2/100
1/1 - 0s - loss: 0.0280 - mse: 0.0280 - val_loss: 0.6056 - val_mse: 0.6056
Epoch 3/100
1/1 - 0s - loss: 0.0271 - mse: 0.0271 - val_loss: 0.6020 - val_mse: 0.6020
Epoch 4/100
1/1 - 0s - loss: 0.0262 - mse: 0.0261 - val_loss: 0.5971 - val_mse: 0.5970
Epoch 5/100
1/1 - 0s - loss: 0.0252 - mse: 0.0252 - val_loss: 0.5913 - val_mse: 0.5913
Epoch 6/100
1/1 - 0s - loss: 0.0243 - mse: 0.0243 - val_loss: 0.5854 - val_mse: 0.5854
Epoch 7/100
1/1 - 0s - loss: 0.0237 - mse: 0.0236 - val_loss: 0.5799 - val_mse: 0.5799
Epoch 8/100
1/1 - 0s - loss: 0.0232 - mse: 0.0232 - val_loss: 0.5753 - val_mse: 0.5753
Epoch 9/100
1/1 - 0s - loss: 0.0229 - mse: 0.0228 - val_loss: 0.5721 - val_mse: 0.5720
Epoch 10/100
1/1 - 0s - loss: 0.0227 - mse: 0.0226 - val_loss: 0.5703 - val_mse: 0.5702
Epoch 11/100
1/1 - 0s - loss: 0.0224 - mse: 0.0224 - val_loss: 0.5701 - val_mse: 0.5701
Epoch 12/100
1/1 - 0s - loss: 0.0221 - ms

In [ ]:
def model_predict(pred_y, target_df,col_name):
    pred_y = model.predict(pred_y)
    target_df[col_name] = pred_y
    return train_df

In [ ]:
 model_predict(pred_y, train_df, 'rating_new')

---


In [41]:
def not_split_pred(target_df, target_feature_names):
    pred_y = {name: target_df[name] for name in target_feature_names}
    return pred_y

In [42]:
pred_y = pred_make(test_df_transform, test_last_feature)

---

In [ ]:
test_scaler_encoder =scaler_encoder(slice_sparse, slice_dense, test_df)

TypeError: 'DataFrame' object is not callable

In [ ]:
slice_sparse

['img_link', 'name', 'processor', 'ram', 'os', 'storage']

In [ ]:
slice_dense

['price(in Rs.)', 'display(in inch)']

In [ ]:
test_df

,img_link,name,price(in Rs.),processor,ram,os,storage,display(in inch),rating,no_of_ratings,no_of_reviews
7,165,91,0.042250,1,13,1,7,0.421053,NaN,NaN,NaN
53,162,0,0.280000,11,17,3,10,0.350877,NaN,NaN,NaN
58,94,62,0.137500,18,16,1,10,0.421053,NaN,NaN,NaN
64,92,116,0.175000,24,0,1,3,0.421053,NaN,NaN,NaN
72,119,165,0.950000,31,8,5,5,0.947368,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
971,165,91,0.042250,1,13,1,7,0.421053,NaN,NaN,NaN
972,204,105,0.200022,19,0,5,10,0.701754,NaN,NaN,NaN
975,189,173,0.462525,24,1,1,3,1.000000,NaN,NaN,NaN
977,134,72,0.118775,18,0,1,10,0.701754,NaN,NaN,NaN


In [ ]:
scaler_encoder(slice_sparse, slice _dense, train_df)

TypeError: 'DataFrame' object is not callable